# Домашнее задание к лекции "Основы веб-скрапинга"


### Обязательная часть

Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

> <дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


In [1]:
#подключаем библиотеки
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import re

In [2]:
# Vers.01 -- date, title, link
def get_habr_posts(query):
    habr_blog = pd.DataFrame()
    for q in query:
        URL = 'https://habr.com/ru/search/'
        params = {
            'q': q
        }
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text)
        articles = soup.find_all('article', class_='tm-articles-list__item')
        for article in articles:
            if article.find(class_="tm-article-snippet__title-link") is None:
                title = article.find(class_="tm-megapost-snippet__link tm-megapost-snippet__card").text
                link = 'https://habr.com' + article.find("a", class_="tm-megapost-snippet__link tm-megapost-snippet__card").get("href")
                date = article.find('time', class_='tm-megapost-snippet__datetime-published').text.strip()
            else:
                title = article.find('h2', class_='tm-article-snippet__title').text
                link = 'https://habr.com' + article.find('h2', class_='tm-article-snippet__title').find('a').get('href')
                date = article.find('span', class_='tm-article-snippet__datetime-published').text.strip()
            row = {'date': date, 'title': title, 'link': link}
            habr_blog = pd.concat([habr_blog, pd.DataFrame([row])]) 
    return habr_blog.reset_index(drop=True)

res = get_habr_posts(['python', 'анализ данных'])
res.drop_duplicates(subset='link')
res

,date,title,link
0,20 января 2022 в 18:37,Курс «Python для инженеров». Старт 3 потока 31...,https://habr.com/ru/company/southbridge/news/t...
1,13 декабря 2021 в 09:00,Жаждущим автоматизации: открытый урок «ChatOps...,https://habr.com/ru/company/southbridge/news/t...
2,21 апреля 2020 в 18:35,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/t/498364/
3,6 июля 2021 в 13:29,Python Community Meetup 8/07: видео и материал...,https://habr.com/ru/company/raiffeisenbank/new...
4,13 января 2022 в 18:35,Открытый урок «Пишем Custom Prometheus Exporte...,https://habr.com/ru/company/southbridge/news/t...
5,4 декабря 2020 в 21:03,Python как компилируемый статически типизирова...,https://habr.com/ru/news/t/531402/
6,8 марта 2022 в 12:13,Вышел мартовский релиз расширения Python для V...,https://habr.com/ru/news/t/654707/
7,3 марта 2020 в 13:22,В начале этого года Python сместил Java и стал...,https://habr.com/ru/company/itsumma/news/t/490...
8,16 ноября 2021 в 16:09,EPAM разработала бесплатный курс по программир...,https://habr.com/ru/company/epam_systems/news/...
9,26 декабря 2022 в 14:24,Positive Technologies открыла доступ по запрос...,https://habr.com/ru/news/t/707698/


### Дополнительная часть (необязательная)

Функция из обязательной части задания должна быть расширена следующим образом:

* кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
* в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:

> <дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [3]:
# Функция для замены нескольких значений
def multiple_replace(target_str, replace_values):
    # получаем заменяемое: подставляемое из словаря в цикле
    for i, j in replace_values.items():
        # меняем все target_str на подставляемое
        target_str = target_str.replace(i, j)
    return target_str

# создаем словарь со значениями и строку, которую будет изменять
replace_values = {'\n': ' ', '\r' : ' '}

In [4]:
# Функция для добавления полного текста статьи
def add_full_text(posts_df):
    i = 0
    for el in posts_df['link']:
        req = requests.get(el).text
        soup = BeautifulSoup(req)
        time.sleep(0.3)
        post_text = soup.find('div', class_=['article-formatted-body article-formatted-body article-formatted-body_version-1', 'article-formatted-body article-formatted-body article-formatted-body_version-2']).text.strip()
        post_text = multiple_replace(post_text, replace_values)
        posts_df.loc[i, 'text'] = post_text
        i += 1
    return posts_df

In [11]:
# Vers.02 -- add full_text, likes and page NEW
def get_habr_posts_full(page, query):
    habr_blog = pd.DataFrame()
    for q in query:
        URL = 'https://habr.com/ru/search/page' + str(page) + '/'
        params = {
            'q': q
        }
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text)
        articles = soup.find_all('article', class_='tm-articles-list__item')
        for article in articles:
            regex = r'Всего голосов.*?(\d+)'
            if article.find(class_="tm-article-snippet__title-link") is None:
                title = article.find(class_="tm-megapost-snippet__link tm-megapost-snippet__card").text
                link = 'https://habr.com' + article.find("a", class_="tm-megapost-snippet__link tm-megapost-snippet__card").get("href")
                date = article.find('time', class_='tm-megapost-snippet__datetime-published').text.strip()
                span_likes = article.find_all('span', class_=['tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating', 
                                                              'tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_appearance-article tm-votes-meter__value_rating'])
            else:
                title = article.find('h2', class_='tm-article-snippet__title').text
                link = 'https://habr.com' + article.find('h2', class_='tm-article-snippet__title').find('a').get('href')
                date = article.find('span', class_='tm-article-snippet__datetime-published').text.strip()
                span_likes = article.find_all('span', class_=['tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating', 
                                                              'tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_appearance-article tm-votes-meter__value_rating'])
            likes = re.findall(regex, str(span_likes))
            likes = int(likes[0])
            row = {'date': date, 'title': title, 'link': link, 'likes': likes}
            habr_blog = pd.concat([habr_blog, pd.DataFrame([row])]) 

    return add_full_text(habr_blog.reset_index(drop=True))
     
res_full = get_habr_posts_full(2, ['python', 'анализ данных'])
res_full.drop_duplicates(subset='link')
res_full

,date,title,link,likes,text
0,14 декабря 2022 в 18:01,Юбилейный поток Python для инженеров,https://habr.com/ru/company/southbridge/news/t...,9,16 января весь Слёрм будет праздновать: старту...
1,16 сентября 2020 в 16:48,Астрономам порекомендовали меньше использовать...,https://habr.com/ru/news/t/519414/,26,Астрономы из Лейденской обсерватории опубликов...
2,15 января 2022 в 19:00,Разработчик предложил устроить коммунистическу...,https://habr.com/ru/news/t/645777/,27,14 января 2022 года разработчик jokteur на Git...
3,18 мая 2022 в 18:33,Слёрм запускает 3-дневный интенсив по Python д...,https://habr.com/ru/company/southbridge/news/t...,7,24-26 июня пройдёт онлайн-интенсив для инженер...
4,18 марта 2022 в 18:31,24 марта Слёрм проведёт открытый урок «Первый ...,https://habr.com/ru/company/southbridge/news/t...,20,"Думаете, написать свою первую программу на Pyt..."
5,22 апреля 2022 в 14:42,TechnoMeetsPython. Онлайн митап о Python-разра...,https://habr.com/ru/news/t/662437/,2,27 апреля в 18:00 собираем питонистов на YouTu...
6,3 августа 2022 в 15:23,Последний поток Python для инженеров в этом году,https://habr.com/ru/company/southbridge/news/t...,8,29 августа мы запускаем 4 поток курса «Python ...
7,29 сентября 2022 в 09:34,Асинхронное программирование на Python для джу...,https://habr.com/ru/company/southbridge/news/t...,15,Как джуну выделиться на фоне таких же новичков...
8,27 сентября 2021 в 09:03,«Разработчикам вход запрещен» или курс «Python...,https://habr.com/ru/company/southbridge/news/t...,13,Полгода назад мы анонсировали первый поток кур...
9,10 декабря 2021 в 13:15,Интенсив для повышения квалификации: как испол...,https://habr.com/ru/company/netologyru/news/t/...,6,"14 декабря в Нетологии пройдёт вебинар, посвящ..."
